# Load tests with K6


In [1]:
import os
from dotenv import load_dotenv

%load_ext autoreload
%autoreload 2

load_dotenv(override=True)

True

## 0. Lookup compatible configs


First, get IE compute instance options


In [2]:
from autobench.utils import ComputeOptionUtil

In [3]:
compute_option_util = ComputeOptionUtil()
compute_option_util.options

,vendor,vendor_status,region,region_label,region_status,id,accelerator,num_gpus,memory_in_gb,gpu_memory_in_gb,instance_type,instance_size,architecture,status,price_per_hour,num_cpus
0,aws,available,us-east-1,N. Virginia,available,aws-us-east-1-nvidia-t4-x1,gpu,1,15,16,nvidia-t4,x1,Nvidia T4,available,0.5,3
1,aws,available,us-east-1,N. Virginia,available,aws-us-east-1-nvidia-t4-x4,gpu,4,192,64,nvidia-t4,x4,Nvidia T4,available,3.0,46
2,aws,available,us-east-1,N. Virginia,available,aws-us-east-1-nvidia-a10g-x1,gpu,1,30,24,nvidia-a10g,x1,Nvidia A10G,available,1.0,6
3,aws,available,us-east-1,N. Virginia,available,aws-us-east-1-nvidia-a10g-x4,gpu,4,186,96,nvidia-a10g,x4,Nvidia A10G,available,5.0,46
4,aws,available,us-east-1,N. Virginia,available,aws-us-east-1-nvidia-a100-x1,gpu,1,145,80,nvidia-a100,x1,Nvidia A100,available,4.0,11
5,aws,available,us-east-1,N. Virginia,available,aws-us-east-1-nvidia-a100-x2,gpu,2,290,160,nvidia-a100,x2,Nvidia A100,available,8.0,22
6,aws,available,us-east-1,N. Virginia,available,aws-us-east-1-nvidia-a100-x4,gpu,4,580,320,nvidia-a100,x4,Nvidia A100,available,16.0,44
7,aws,available,us-east-1,N. Virginia,available,aws-us-east-1-nvidia-a100-x8,gpu,8,1160,640,nvidia-a100,x8,Nvidia A100,available,32.0,88
8,aws,available,us-east-1,N. Virginia,available,aws-us-east-1-nvidia-l4-x4,gpu,4,185,96,nvidia-l4,x4,Nvidia L4,available,3.8,47
9,aws,available,us-east-1,N. Virginia,available,aws-us-east-1-nvidia-l4-x1,gpu,1,30,24,nvidia-l4,x1,Nvidia L4,available,0.8,7


In [4]:
compute_option_util.options.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   vendor            21 non-null     object 
 1   vendor_status     21 non-null     object 
 2   region            21 non-null     object 
 3   region_label      21 non-null     object 
 4   region_status     21 non-null     object 
 5   id                21 non-null     object 
 6   accelerator       21 non-null     object 
 7   num_gpus          21 non-null     int64  
 8   memory_in_gb      21 non-null     int64  
 9   gpu_memory_in_gb  21 non-null     int64  
 10  instance_type     21 non-null     object 
 11  instance_size     21 non-null     object 
 12  architecture      21 non-null     object 
 13  status            21 non-null     object 
 14  price_per_hour    21 non-null     float64
 15  num_cpus          21 non-null     int64  
dtypes: float64(1), int64(4), object(11)
memory usa

User specifies their desired inputs


In [5]:
VENDOR = "aws"
REGION = "us-east-1"
GPU_TYPES = ["nvidia-a10g", "nvidia-l4"]

In [6]:
possible_instances = compute_option_util.get_instance_details(
    vendor=VENDOR, region=REGION, gpu_types=GPU_TYPES
)

In [7]:
possible_instances

[{'vendor': 'aws',
  'vendor_status': 'available',
  'region': 'us-east-1',
  'region_label': 'N. Virginia',
  'region_status': 'available',
  'id': 'aws-us-east-1-nvidia-a10g-x1',
  'accelerator': 'gpu',
  'num_gpus': 1,
  'memory_in_gb': 30,
  'gpu_memory_in_gb': 24,
  'instance_type': 'nvidia-a10g',
  'instance_size': 'x1',
  'architecture': 'Nvidia A10G',
  'status': 'available',
  'price_per_hour': 1.0,
  'num_cpus': 6},
 {'vendor': 'aws',
  'vendor_status': 'available',
  'region': 'us-east-1',
  'region_label': 'N. Virginia',
  'region_status': 'available',
  'id': 'aws-us-east-1-nvidia-a10g-x4',
  'accelerator': 'gpu',
  'num_gpus': 4,
  'memory_in_gb': 186,
  'gpu_memory_in_gb': 96,
  'instance_type': 'nvidia-a10g',
  'instance_size': 'x4',
  'architecture': 'Nvidia A10G',
  'status': 'available',
  'price_per_hour': 5.0,
  'num_cpus': 46},
 {'vendor': 'aws',
  'vendor_status': 'available',
  'region': 'us-east-1',
  'region_label': 'N. Virginia',
  'region_status': 'available

Then, check if model will work on any of the desired instances, and if so, get each TGI config


In [12]:
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

viable_instances = compute_option_util.get_viable_instance_configs(
    model_id=model_id, instances=possible_instances
)

In [13]:
viable_instances

[{'tgi_config': {'model_id': 'meta-llama/Meta-Llama-3-8B-Instruct',
   'max_batch_prefill_tokens': 8192,
   'max_input_length': 6000,
   'max_total_tokens': 6144,
   'num_shard': 1,
   'quantize': None,
   'estimated_memory_in_gigabytes': 23.01},
  'instance_config': {'vendor': 'aws',
   'vendor_status': 'available',
   'region': 'us-east-1',
   'region_label': 'N. Virginia',
   'region_status': 'available',
   'id': 'aws-us-east-1-nvidia-a10g-x1',
   'accelerator': 'gpu',
   'num_gpus': 1,
   'memory_in_gb': 30,
   'gpu_memory_in_gb': 24,
   'instance_type': 'nvidia-a10g',
   'instance_size': 'x1',
   'architecture': 'Nvidia A10G',
   'status': 'available',
   'price_per_hour': 1.0,
   'num_cpus': 6}},
 {'tgi_config': {'model_id': 'meta-llama/Meta-Llama-3-8B-Instruct',
   'max_batch_prefill_tokens': 32768,
   'max_input_length': 6000,
   'max_total_tokens': 6144,
   'num_shard': 4,
   'quantize': None,
   'estimated_memory_in_gigabytes': 98.36},
  'instance_config': {'vendor': 'aws',


In [14]:
viable_instances[0]

{'tgi_config': {'model_id': 'meta-llama/Meta-Llama-3-8B-Instruct',
  'max_batch_prefill_tokens': 8192,
  'max_input_length': 6000,
  'max_total_tokens': 6144,
  'num_shard': 1,
  'quantize': None,
  'estimated_memory_in_gigabytes': 23.01},
 'instance_config': {'vendor': 'aws',
  'vendor_status': 'available',
  'region': 'us-east-1',
  'region_label': 'N. Virginia',
  'region_status': 'available',
  'id': 'aws-us-east-1-nvidia-a10g-x1',
  'accelerator': 'gpu',
  'num_gpus': 1,
  'memory_in_gb': 30,
  'gpu_memory_in_gb': 24,
  'instance_type': 'nvidia-a10g',
  'instance_size': 'x1',
  'architecture': 'Nvidia A10G',
  'status': 'available',
  'price_per_hour': 1.0,
  'num_cpus': 6}}

## 1. Deploy LLM with TGI on Inference Endpoints


In [15]:
from autobench.deploy import IEDeployment
from autobench.utils import TGIConfig, ComputeInstanceConfig, ComputeOptionUtil

/Users/andrewreed/Documents/success_projects/auto-bench/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:
viable_instance = viable_instances[0]

In [17]:
viable_instance

{'tgi_config': {'model_id': 'meta-llama/Meta-Llama-3-8B-Instruct',
  'max_batch_prefill_tokens': 8192,
  'max_input_length': 6000,
  'max_total_tokens': 6144,
  'num_shard': 1,
  'quantize': None,
  'estimated_memory_in_gigabytes': 23.01},
 'instance_config': {'vendor': 'aws',
  'vendor_status': 'available',
  'region': 'us-east-1',
  'region_label': 'N. Virginia',
  'region_status': 'available',
  'id': 'aws-us-east-1-nvidia-a10g-x1',
  'accelerator': 'gpu',
  'num_gpus': 1,
  'memory_in_gb': 30,
  'gpu_memory_in_gb': 24,
  'instance_type': 'nvidia-a10g',
  'instance_size': 'x1',
  'architecture': 'Nvidia A10G',
  'status': 'available',
  'price_per_hour': 1.0,
  'num_cpus': 6}}

In [18]:
tgi_config = TGIConfig(**viable_instance["tgi_config"])
instance_config = ComputeInstanceConfig(**viable_instance["instance_config"])

In [19]:
tgi_config.env_vars

{'MAX_BATCH_PREFILL_TOKENS': '8192',
 'MAX_INPUT_LENGTH': '6000',
 'MAX_TOTAL_TOKENS': '6144',
 'NUM_SHARD': '1',
 'MODEL_ID': '/repository'}

In [20]:
deployment = IEDeployment(tgi_config=tgi_config, instance_config=instance_config)

In [21]:
print(deployment.tgi_config)

TGIConfig(model_id='meta-llama/Meta-Llama-3-8B-Instruct', max_batch_prefill_tokens=8192, max_input_length=6000, max_total_tokens=6144, num_shard=1, quantize=None, estimated_memory_in_gigabytes=23.01)


In [22]:
deployment.endpoint_name

'autobench'

In [24]:
deployment.deploy_endpoint()

Creating inference endpoint...
Endpoint created successfully: https://v1wfvykisrysn6e0.us-east-1.aws.endpoints.huggingface.cloud


In [25]:
deployment.endpoint.url

'https://v1wfvykisrysn6e0.us-east-1.aws.endpoints.huggingface.cloud'

## 2. Prepare ShareGPT data for realisitic inference workload


In [28]:
import json
import datasets
import tqdm

In [29]:
dataset = datasets.load_dataset(
    "Open-Orca/slimorca-deduped-cleaned-corrected", split="train"
)

# Select only the first 2k conversations
max = min(2000, len(dataset))
conversations = []

for item in tqdm.tqdm(dataset, total=max):
    conv = item.get("conversations")
    if conv and conv[0]["from"] == "system":
        # Get only the initial user message
        conv = conv[1:2]
        conversations.append(conv)

        if len(conversations) >= max:
            break


with open("..benchmark_data/small.json", "w") as f:
    data = json.dump(conversations, f, indent=4)

100%|█████████▉| 1999/2000 [00:00<00:00, 47939.97it/s]


## 2. Run K6 Load Test


In [26]:
# If necessary, install the openai Python library by running
# pip install openai

import requests
from huggingface_hub import get_token


API_URL = f"{deployment.endpoint.url}" + "/v1/chat/completions"
headers = {
    "Accept": "application/json",
    "Authorization": f"Bearer {get_token()}",
    "Content-Type": "application/json",
}


def query(payload):
    response = requests.post(API_URL, headers=headers, json=payload)
    return response


output = query(
    {
        "messages": [{"role": "user", "content": "How old is Statute of Liberty?"}],
        "max_tokens": 150,
        "model": "tgi",
    }
)

In [27]:
print(output.content)

b'{"object":"chat.completion","id":"","created":1724350025,"model":"/repository","system_fingerprint":"2.2.1-dev0-sha-358ceb6","choices":[{"index":0,"message":{"role":"assistant","content":"The Statue of Liberty, also known as Lady Liberty, was dedicated on October 28, 1886. It was designed by French sculptor Fr\xc3\xa9d\xc3\xa9ric Auguste Bartholdi and built by Gustave Eiffel. The statue was a gift from the people of France to the people of the United States to commemorate the 100th anniversary of American independence."},"logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":18,"completion_tokens":75,"total_tokens":93}}'


In [28]:
print(output.headers)

{'Date': 'Thu, 22 Aug 2024 18:07:05 GMT', 'Content-Type': 'application/json', 'Content-Length': '626', 'Connection': 'keep-alive', 'x-compute-type': '1-nvidia-a10g', 'x-compute-time': '2.556334947', 'x-compute-characters': '146', 'x-total-time': '2556', 'x-validation-time': '0', 'x-queue-time': '0', 'x-inference-time': '2555', 'x-time-per-token': '34', 'x-prompt-tokens': '19', 'x-generated-tokens': '75', 'vary': 'origin, access-control-request-method, access-control-request-headers, origin, access-control-request-method, access-control-request-headers', 'access-control-allow-origin': '*', 'x-proxied-host': 'http://10.101.113.22', 'x-proxied-path': '/v1/chat/completions', 'x-request-id': 'TXh2Sy', 'access-control-allow-credentials': 'true'}


In [29]:
from autobench.k6 import K6Config, K6ConstantArrivalRateExecutor, K6Benchmark

In [30]:
executor = K6ConstantArrivalRateExecutor(
    pre_allocated_vus=10, rate_per_second=1, duration="20s"
)

In [31]:
config = K6Config(
    host=deployment.endpoint.url,
    executor=executor,
    data_file="small.json",
)

In [32]:
print(config)

K6Config(url=https://v1wfvykisrysn6e0.us-east-1.aws.endpoints.huggingface.cloud executor=<autobench.k6.K6ConstantArrivalRateExecutor object at 0x168febd60> data_file=small.json)


In [33]:
config.executor.variables

{'pre_allocated_vus': 10,
 'rate': 1,
 'duration': '20s',
 'host': 'https://v1wfvykisrysn6e0.us-east-1.aws.endpoints.huggingface.cloud',
 'data_file': 'small.json',
 'data_path': '/Users/andrewreed/Documents/success_projects/auto-bench/autobench/benchmark_data',
 'temp_dir': '/var/folders/w0/6t9rxkj97rv47l9sc0q22yth0000gn/T/autobench_d_z_xlud_k6_results'}

In [34]:
benchmark = K6Benchmark(config=config, output_dir="./results")

In [35]:
benchmark.output_dir

'./results'

In [36]:
config.host

'https://v1wfvykisrysn6e0.us-east-1.aws.endpoints.huggingface.cloud'

In [37]:
benchmark.run()


          /\      |‾‾| /‾‾/   /‾‾/   
     /\  /  \     |  |/  /   /  /    
    /  \/    \    |     (   /   ‾‾\  
   /          \   |  |\  \ |  (‾)  | 
  / __________ \  |__| \__\ \_____/ .io

     execution: local
        script: /var/folders/w0/6t9rxkj97rv47l9sc0q22yth0000gn/T/autobench_2cccx0phk6_script
        output: json (/var/folders/w0/6t9rxkj97rv47l9sc0q22yth0000gn/T/autobench_d_z_xlud_k6_results/results.json)

     scenarios: (100.00%) 1 scenario, 10 max VUs, 50s max duration (incl. graceful stop):
              * load_test: 1.00 iterations/s for 20s (maxVUs: 10, gracefulStop: 30s)


running (01.0s), 01/10 VUs, 0 complete and 0 interrupted iterations
load_test   [   5% ] 01/10 VUs  01.0s/20s  1.00 iters/s

running (02.0s), 02/10 VUs, 0 complete and 0 interrupted iterations
load_test   [  10% ] 02/10 VUs  02.0s/20s  1.00 iters/s

running (03.0s), 03/10 VUs, 0 complete and 0 interrupted iterations
load_test   [  15% ] 03/10 VUs  03.0s/20s  1.00 iters/s

running (04.0s), 03/10 

In [38]:
benchmark.get_summary_path()

'./results.summary.json'

aScheduler design:

Needs to:

1. Take in:
   - model_id
   - GPU types
   - Optionally, provider (AWS/GCP)
   - Optionally, specify (MAX_INPUT_TOKENS, MAX_TOTAL_TOKENS, MAX_BATCH_PREFILL_TOKENS, MAX_BATCH_TOTAL_TOKENS)
2. Determine which instances can fit the desired model
3. Iterate through instances to:
   a. deploy
   b. benchmark
   c. report results

Runner design:

1. Take in model_id, instance, TGI config
2. Deploy this
3. Shut down / pause instance
